In [3]:
from agential.prompting.standard.prompting import Standard
import os
import json
import pickle

from agential.prompting.standard.prompts import (
    STANDARD_INSTRUCTION_AMBIGNQ, 
    STANDARD_INSTRUCTION_FEVER, 
    STANDARD_INSTRUCTION_GSM8K,  
    STANDARD_INSTRUCTION_HOTPOTQA, 
    STANDARD_INSTRUCTION_SVAMP, 
    STANDARD_INSTRUCTION_TRIVIAQA,
    STANDARD_INSTRUCTION_TABMWP,
    STANDARD_INSTRUCTION_HUMANEVAL,
    STANDARD_INSTRUCTION_MBPP,
)
from agential.core.fewshots.ambignq import AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.fever import FEVER_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.svamp import SVAMP_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.triviaqa import TRIVIAQA_FEWSHOT_EXAMPLES_DIRECT
from agential.core.fewshots.tabmwp import TABMWP_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT
from agential.core.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_POT

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

from agential.core.llm import LLM

import wandb
wandb.login()

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

with open('../data/hotpotqa/hotpot_dev_v1_simplified_s42_sample500.json', 'r') as file:
    data = json.load(file)

In [ ]:
from agential.eval.metrics.classification import EM, f1, precision, recall

# Log numbers (EM, f1, precission, recall)
# Log/Save artifacts to your runs
# Optional, how to log final metrics 

method = Standard(
        llm=llm,
        benchmark="hotpotqa",
    )

num_correct = 0 
samples = 20

wandb.init("standard_hotpotqa")

for i in data[:samples]:

    question = i["question"]
    answer = i["answer"]

    out = method.generate(
        question=question,
        key=answer
    )
    is_correct = EM(out.answer, answer)
    precision_out = precision(out.answer, answer)
    recall_out = recall(out.answer, answer)
    f1_out = f1(out.answer, answer)

    num_correct += int(is_correct)

wandb.finish()